# Using Arcee-SuperNova-AWQ on SageMaker through Model Packages

---

**If you already deployed the model package with CloudFormation, the AWS CLI or directly in the SageMaker console, please use the [sample-notebook-all-models-existing-sagemaker-endpoint.ipynb](sample-notebook-all-models-existing-sagemaker-endpoint.ipynb) notebook instead.**

---

This sample notebook shows you how to deploy Arcee SuperNova using Amazon SageMaker. Here, we use a 4-bit version quantized with [AutoAWQ](https://github.com/casper-hansen/AutoAWQ), small enough to run on cost-effective 4-GPU instances of the g5 and g6 families.

Arcee-SuperNova is an advanced general language model developed by Arcee AI, offering superior performance, customization, and security compared to traditional closed-source models. It excels in instruction following and human preference scores, outperforming Llama 70B-Instruct, as well Llama 405B, Claude Sonnet 3.5 and GPT-4o in many general benchmarks. SuperNova also provides businesses with the option for deep customization on their data and use cases, seamless integration into existing systems, and top-tier security & compliance. It's ideal for industries with stringent security requirements, data-driven companies, and businesses seeking cost efficiency, offering up to 95% cost reduction compared to closed-source models.

## Use cases

* Ideal for industries with stringent security requirements (finance, healthcare, government)
* Perfect for data-driven companies requiring tailored AI models
* Suitable for businesses seeking cost efficiency without compromising performance
* Use cases include customer service, content creation, data analysis, R&D, legal & compliance, and education & training
  
## Pre-requisites
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.

## Contents
1. [Select model package](#1.-Select-model-package)

2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
    1. [Define the endpoint configuration](#A.-Define-the-endpoint-configuration)
    2. [Create the endpoint](#B.-Create-the-endpoint)
    3. [Define a test payload](#C.-Define-a-test-payload)
    4. [Perform real-time inference](#D.-Perform-real-time-inference)
    5. [Visualize output](#E.-Visualize-output)

3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Delete the endpoint](#B.-Delete-the-endpoint)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select the model package
Confirm that you received this notebook from model catalog on SageMaker AWS Management Console.

In [ ]:
model_package_map = {
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Tokyo
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Seoul
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Mumbai
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Singapore
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Sydney
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Canada Central
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Frankfurt
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Stockholm
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Ireland
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # London
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Paris
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # São Paulo
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # N. Virginia
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Ohio
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # N. California
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/arcee-supernova-v1-awq-tgi-mar-9993ff88ff823eccb942ae8714499cb2",  # Oregon
}

In [ ]:
import datetime
import json
import pprint

import boto3
import sagemaker
from IPython.display import Markdown, display
from sagemaker import ModelPackage, get_execution_role

In [ ]:
region = boto3.Session().region_name

In [ ]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()
runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

In this example, we're deploying Arcee-SuperNova on a SageMaker real-time endpoint hosted on a GPU instance. If you need general information on real-time inference with Amazon SageMaker, please refer to the SageMaker [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

The endpoint runs a Hugging Face [Deep Learning Container](https://huggingface.co/docs/sagemaker/index), powered by the Hugging Face [Text Generation Inference](https://huggingface.co/docs/text-generation-inference/index) Server (TGI). TGI enables high-performance text generation for the most popular open-source language models. 

The endpoint configuration focuses on cost efficiency. It uses a [g5.12xlarge](https://aws.amazon.com/ec2/instance-types/g5/) instance. This instance has four NVIDIA A10G GPUs, with a total of 96 GB of GPU RAM. This is enough to load the 4-bit model with a context size of 32K.

#### OpenAI compatibility

We enable the [OpenAI Messages API](https://huggingface.co/docs/text-generation-inference/messages_api) available in TGI. This will allow you to invoke the endpoint in the same way you would invoke an OpenAI model. Likewise, the output format will be identical to the OpenAI models.

### A. Define the endpoint configuration

In [ ]:
model_name = "arcee-supernova-awq-v1"
real_time_inference_instance_type = "ml.g5.12xlarge"

In [ ]:
if region not in model_package_map.keys():
    raise "UNSUPPORTED REGION"

model_package_arn = model_package_map[region]

### B. Create the endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# create a unique endpoint name
timestamp = "{:%Y-%m-%d-%H-%M-%S}".format(datetime.datetime.now())
endpoint_name = f"{model_name}-{timestamp}"
print(f"Deploying endpoint {endpoint_name}")

# deploy the model
response = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name,
    model_data_download_timeout=1800,
    container_startup_health_check_timeout=900,
)

Once the endpoint is in service, you will be able to perform real-time inference.

### C. Define a test payload

In [ ]:
model_sample_input = {
    "model": "tgi",
    "messages": [
        {"role": "system", "content": "You are a friendly and helpful AI assistant."},
        {
            "role": "user",
            "content": "Suggest 5 names for a new neighborhood pet food store. Names should be short, fun, easy to remember, and respectful of pets. \
        Explain why customers would like them.",
        },
    ],
    "max_tokens": 1024,
}

### D. Perform real-time inference

In [ ]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(model_sample_input),
)

output = json.loads(response["Body"].read().decode("utf8"))

### E. Visualize output

We can print the raw JSON output in OpenAI format.

In [ ]:
pprint.pprint(output)

We can also print the generated output with Markdown formatting.

In [ ]:
display(Markdown(output["choices"][0]["message"]["content"]))

Here are some more examples. Please feel free to tweak them and add your own!

In [ ]:
prompt = """Please write a friendly marketing pitch for a new SaaS AI platform called Arcee Cloud.
We will send this pitch by email to business and technical decision-makers, so make it sound exciting yet professional.
The contact email is sales@arcee.ai. Feel free to use emojis as appopriate.
Arcee Cloud makes it simple for enterprise users to tailor open-source small language models to their own domain knowledge,
in order to build high-quality, cost-effective and secure AI solutions."""

model_sample_input = {
    "model": "tgi",
    "messages": [
        {
            "role": "system",
            "content": "You are a friendly and helpful Marketing Manager working at Arcee.ai.",
        },
        {"role": "user", "content": prompt},
    ],
    "max_tokens": 1024,
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(model_sample_input),
)

output = json.loads(response["Body"].read().decode("utf8"))
display(Markdown(output["choices"][0]["message"]["content"]))

In [ ]:
model_sample_input = {
    "messages": [
        {
            "role": "system",
            "content": "As a friendly technical assistant engineer, answer the question in detail.",
        },
        {"role": "user", "content": "Why are transformers better models than LSTM?"},
    ],
    "max_tokens": 1024,
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(model_sample_input),
)

output = json.loads(response["Body"].read().decode("utf8"))
display(Markdown(output["choices"][0]["message"]["content"]))

In [ ]:
model_sample_input = {
    "model": "tgi",
    "messages": [
        {
            "role": "system",
            "content": "You are Darlene, a friendly and helpful salesperson \
        working at Crystal River Classic Bikes, a classic motorcycle dealership in central Florida.",
        },
        {
            "role": "user",
            "content": "Using English, write a personalized customer email to get \
        them to sign up for a test ride on the new 2025 motorcycles that are visible at the dealership. \
        Tone should be warm and personal, make sure to weave in the customer information below. \
        Wyatt, your chief mechanic and road captain, has just won the 2024 State Award for Best Mechanic. \
        \
        Customer information:\
        - name: Julien \
        - last visit: 6 months ago for bike service \
        - Owns 2 bikes, a 2002 sporty bike and a 2007 cruiser \
        ",
        },
    ],
    "max_tokens": 1024,
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(model_sample_input),
)

output = json.loads(response["Body"].read().decode("utf8"))
display(Markdown(output["choices"][0]["message"]["content"]))

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

## 4. Clean-up

Please don't forget to run the cells below to delete all resources and avoid unecessary charges.

### A. Delete the endpoint

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()

Thank you for trying out Arcee SuperNova on SageMaker. We have only scratched the surface of what you can do with this model.

We'd be happy to hear from you, learn more about your use case, and help you build your next AI-driven solution. Please reach out to julien@arcee.ai.